In [2]:
!pip install sentinelhub

In [3]:
!git clone https://github.com/sentinel-hub/eo-learn.git
!git clone https://github.com/sentinel-hub/water-observatory-backend.git

Cloning into 'eo-learn'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (427/427), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 10451 (delta 222), reused 281 (delta 145), pack-reused 10024
Receiving objects: 100% (10451/10451), 470.90 MiB | 27.16 MiB/s, done.
Resolving deltas: 100% (6381/6381), done.
Checking out files: 100% (360/360), done.
fatal: destination path 'water-observatory-backend' already exists and is not an empty directory.


In [4]:

%cd eo-learn
!python install_all.py '-e'

/content/eo-learn
Obtaining file:///content/eo-learn/core
  Attempting uninstall: eo-learn-core
    Found existing installation: eo-learn-core 0.8.0
    Uninstalling eo-learn-core-0.8.0:
      Successfully uninstalled eo-learn-core-0.8.0
  Running setup.py develop for eo-learn-core
Obtaining file:///content/eo-learn/coregistration
  Attempting uninstall: eo-learn-coregistration
    Found existing installation: eo-learn-coregistration 0.8.0
    Uninstalling eo-learn-coregistration-0.8.0:
      Successfully uninstalled eo-learn-coregistration-0.8.0
  Running setup.py develop for eo-learn-coregistration
Obtaining file:///content/eo-learn/features
  Attempting uninstall: eo-learn-features
    Found existing installation: eo-learn-features 0.8.0
    Uninstalling eo-learn-features-0.8.0:
      Successfully uninstalled eo-learn-features-0.8.0
  Running setup.py develop for eo-learn-features
Obtaining file:///content/eo-learn/geometry
  Attempting uninstall: eo-learn-geometry
    Found existin

In [5]:
%cd ..
!pip install --upgrade pip
!pip install jupyter
!pip install recordclass



!pip install tornado==5.1.1



!wget https://raw.githubusercontent.com/mlubej/water_detection_notebook/master/water_level_extraction.ipynb

/content
--2020-12-02 18:41:27--  https://raw.githubusercontent.com/mlubej/water_detection_notebook/master/water_level_extraction.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-12-02 18:41:27 ERROR 404: Not Found.



## Le notebook n'est plus disponible ....

In [6]:
%cd content

[Errno 2] No such file or directory: 'content'
/content


In [7]:
# set the autoreload and the inline plotting for matplotlib
!pip install eo-learn
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data manipulation
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# image manipulations
from skimage.filters import threshold_otsu, sobel
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, disk

# GIS related
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import shape, MultiPolygon, Polygon

# eo-learn related
from eolearn.core import EOTask, EOPatch, LinearWorkflow, Dependency, FeatureType, LoadFromDisk, SaveToDisk
from eolearn.io import S2L1CWCSInput 
from eolearn.mask import AddCloudMaskTask, AddValidDataMaskTask, get_s2_pixel_cloud_detector
from eolearn.features import SimpleFilterTask
from eolearn.geometry import VectorToRaster

# Sentinel Hub
from sentinelhub import BBox, CRS

# water observatory backend
import sys
sys.path.append('./water-observatory-backend/src')
#from visualisation import plot_water_body
from geom_utils import get_bbox
from s2_water_extraction import get_water_level_optical
from visualisation import draw_multi, draw_poly

# other
import urllib.request as request
import json
from datetime import datetime
from shapely.wkt import loads

ImportError: ignored

## Cette partie marche

In [ ]:


INSTANCE_ID = '5f6bcc03-4ebc-4c89-8bf3-802332444ae5'  # In case you put instance ID into configuration file you can leave this unchanged

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import datetime
import numpy as np

import matplotlib.pyplot as plt
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

from sentinelhub import CustomUrlParam


# Tip: if you want to insert the coordinates from google, you will need to set
# the first two coordinates for the upper left corner (-122.41, 39.31)
# and second two (-122.75, 39.55) will refer to lower right corner of the box
# Lastly: lat long from Google maps needs to be switched around (e.g. for lower corner 
# google maps will give you '39.55, -122.75'; you need to switch out around to read -122.75, 39.55)

betsiboka_coords_wgs84 = [-122.41, 39.31, -122.75, 39.55]
betsiboka_bbox2 = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)


my_url = 'https://raw.githubusercontent.com/sentinel-hub/custom-scripts/master/sentinel-2/markuse_fire/script.js'
# https://raw.githubusercontent.com/sentinel-hub/custom-scripts/master/sentinel-2/nbr/script.js

evalscripturl_wms_request = WmsRequest(layer='TRUE-COLOR-S2-L1C', # Layer parameter can be any existing layer
                                       bbox=betsiboka_bbox2,
                                       time='2018-08-28',
                                       width=512,
                                       instance_id=INSTANCE_ID,
                                       custom_url_params={CustomUrlParam.EVALSCRIPTURL: my_url})

evalscripturl_wms_data = evalscripturl_wms_request.get_data()
plot_image(evalscripturl_wms_data[0])